In [17]:
from scipy.stats import norm
from tensorflow import keras
import numpy as np
import pandas as pd
import timeit
data = pd.read_csv('.\\data\\calls_OMrates587569.csv')
# columns: 'strike', 'moneyness', 'rate', 'volatility', 'days_to_maturity', 'contract_price'

In [18]:
len(data)

587569

In [19]:
# Black-Scholes formula: C = N(d1)S - N(d2)*K*e^{-r*T},
# where N is Normal CDF, and
# d1 = ( (log M) + (r+0.5*var^2)*t )/(var*T**0.5),
# d2 = d1 - var*T**0.5, for M = S/K so also
# C = K*( N(d1)*M - N(d2)*e^{-r*T} ).

def black_scholes(df): # Returns analytical B-S prices for each row of inputs in df
    N      = lambda vec: norm.cdf(vec)
    mats   = df.days_to_maturity / 365
    d1_num = np.log(df.moneyness) + (df.rate + 0.5 * df.volatility ** 2) * mats
    d1_den = df.volatility * mats ** 0.5
    d1     = d1_num / d1_den
    d2     = d1 - d1_den
    return df.strike * ( N(d1) * df.moneyness - N(d2) * np.exp(-df.rate * mats) )

data['bs_prices'] = black_scholes(data)
bs_mse  = np.sum( (data.bs_prices - data.contract_price) ** 2 ) / len(data)
bs_mape = 100 * sum( np.abs(data.bs_prices - data.contract_price) / data.contract_price ) / len(data)

In [20]:
print(f'bs_mse  {bs_mse:.4f}')   # Average BS squared error is ~0.15
print(f'bs_mape {bs_mape:.4f}')  # Average absolute percentage error is almost 100%(!). We can do much better...

bs_mse  0.1496
bs_mape 98.7203


The below is NOT the test loss as this is evaluating on all data, including training data, so only to visualise some example prices and estimates. The reported loss is the last 3 epochs' averaged validation error reported as a loss metric by TensorFlow.

In [21]:
logdir = '.\\trained_models_and_logs\\accuracy\\saved_model_accuracy'
saved_model = keras.models.load_model(logdir)
data['nn_prices'] = saved_model.predict(data[data.columns[:5]].to_numpy())
nn_mse  = np.sum( (data.nn_prices - data.contract_price) ** 2 ) / len(data)
nn_mape = 100 * sum( np.abs(data.nn_prices - data.contract_price) / data.contract_price ) / len(data)

In [24]:
print(f'nn_mse  {nn_mse:.4f}')   # Average NN squared error is ~0.0072
print(f'nn_mape {nn_mape:.4f}')  # Average absolute percentage error is 30.8%

nn_mse  0.0072
nn_mape 30.8152


In [25]:
data.tail() # Some example values

,strike,moneyness,rate,volatility,days_to_maturity,contract_price,bs_prices,nn_prices
587564,8.3,0.984217,0.025156,0.103653,29.0,0.0640,0.049331,0.071956
587565,8.3,0.991928,0.024910,0.091511,23.0,0.0850,0.051747,0.078279
587566,8.4,0.975833,0.025798,0.080632,57.0,0.1000,0.041590,0.076956
587567,8.5,0.964353,0.025798,0.080632,57.0,0.0745,0.021583,0.063407
587568,8.4,0.973095,0.025731,0.113304,38.0,0.0600,0.045252,0.066345
